# **处理最终测试集，因为测试集每行最后都多了一个"\t"**

In [1]:
import os
data_path="/home/kesci/input/bytedance/bytedance_contest.final_2.csv"
data_test="/home/kesci/test2.csv"
fw=open(data_test,"w")
all_data=open(data_path)
for line in all_data:
    fw.write(line.strip()+"\n")
fw.close()

# 生成128维word2vec词向量以及128维FastText词向量，并生成word2id文件

In [ ]:
# 生成word2vec词向量 128维
import gensim
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
import multiprocessing
from random import random

path = get_tmpfile("/home/kesci/word2vec.model")  # 创建临时文件

sentences=[] #句子的list
f = open("/home/kesci/input/bytedance/train_final.csv")
for line in f: 
    line = line.strip().split(",")
    if random()<0.1:  #以0.1的概率随机抽取句子title和query
        sentences.append(line[1])
        sentences.append(line[3])
f.close()

f=open("/home/kesci/input/bytedance/test_final_part1.csv")
for line in f:
    line = line.strip().split(",")
    sentences.append(line[1])
    sentences.append(line[3])
f.close()


sentences=[]
f = open("/home/kesci/test2.csv")
for line in f:
    line = line.strip().split(",")
    sentences.append(line[1])
    sentences.append(line[3])
f.close()
print(len(sentences))

class MySentences(object):
    def __init__(self, sentences):
        self.sentences = sentences

    def __iter__(self):
        print("djkbakjs")
        for line in self.sentences:
            yield line.split()
        
all_sentences = MySentences(sentences)
model = Word2Vec(all_sentences, size=128, window=5, min_count=5, workers=4)
model.save("/home/kesci/word2vec.model")

# 生成FastText词向量 128维
from gensim.models import FastText
path = get_tmpfile("./data/fast_w2v.model")  # 创建临时文件


class MySentences(object):
    def __init__(self, sentences):
        self.sentences = sentences

    def __iter__(self):
        print("djkbakjs")
        for line in self.sentences:
            yield line.split()


all_sentences = MySentences(sentences)
model = FastText(all_sentences, size=128, window=5, min_count=5, workers=4)
model.save("./data/fast_w2v.model")

# 生成word2id文件，统计每个词的词频，从大到小排序之后，并且去除词频小于5的词
my_dict={}
for ju in sentences:
    for w in ju.split():
        my_dict.setdefault(w,0)
        my_dict[w]+=1
p_d=sorted(my_dict.items(),key=lambda item:item[1],reverse=True)
f_w=open("./data/word2id.txt","w")
idx=1
sum_word=0
for i in p_d:
    if i[1]>=5:
        sum_word+=1
        f_w.write(i[0]+" "+str(idx)+"\n")
        idx += 1
print(sum_word)
f_w.close()

# **分割数据，将10亿数据按顺序平均分成20份**

In [ ]:
import os
data_path="/home/kesci/input/bytedance/train_final.csv"
split_data="/home/kesci/order_data"
length=1000000000 

num_kuai=20
kuai_size=length//num_kuai

all_data=open(data_path)
fw=open(os.path.join(split_data,"0.csv"),"w")
kuai_num=0
for i,line in enumerate(all_data,1):
    fw.write(line.strip()+"\n") # 因为训练集每行最后也有\t，所以需要用line.strip()去除\t
    if i%kuai_size==0:
        kuai_num+=1
        fw.close()
        if kuai_num==20:
            break
        fw = open(os.path.join(split_data,str(kuai_num)+".csv"),"w")
fw.close()

# 分割最后5000万数据，生成本地的验证集，就是最后的5万数据。

In [ ]:
f=open("/home/kesci/order_data/19.csv")
fw=open("/home/kesci/19_train.csv","w")
row=0
for i in f:
    fw.write(i)
    row+=1
    if row==49950000:
        break
fw.close()

fw=open("./data/eval_data/19_eval.csv","w")
for i in f:
    fw.write(i)
fw.close()
f.close()

# 针对前9亿数据和最后4995万数据，生成打乱后的数据，用于训练esim模型，后5万是验证集，18.csv本来打算用来融合模型，后来效果不好，就没用。

In [ ]:
import os
filename=os.listdir("/home/kesci/order_data")
fs=[open("/home/kesci/order_data/"+f) for f in filename if f!="18.csv" and f!="19.csv"]
fs.append(open("/home/kesci/19_train.csv"))
import random
fw=open("/home/kesci/shuffle_data.csv","w")
while len(fs):
    temp_index=random.randint(0,len(fs)-1) #每次随机从19份数据中
    temp_str=fs[temp_index].readline()
    if temp_str:
        fw.write(temp_str)
    else:
        fs.remove(fs[temp_index])
fw.close()

# 统计所有词的文档频率，即所有词在所有title和query里面出现的次数，包括最终测试集和复赛第一个测试集

In [ ]:
from collections import Counter
import pandas as pd
import gc

data_95 = open("/home/kesci/input/bytedance/train_final.csv")
count_set = set()

for line in data_95:
    line = line.strip().split(',')
    count_set.add(line[1])
    count_set.add(line[3])
data_95.close()

data_95 = open("/home/kesci/input/bytedance/bytedance_contest.final_2.csv")

for line in data_95:
    line = line.strip().split(',')
    count_set.add(line[1])
    count_set.add(line[3])
data_95.close()

data_95 = open("/home/kesci/input/bytedance/test_final_part1.csv")
for line in data_95:
    line = line.strip().split(',')
    count_set.add(line[1])
    count_set.add(line[3])
data_95.close()
print(len(count_set)) 
doc_sum_fre=len(count_set) #记录文档的总数 237412321
word_sens = {} #key是词，value是词出现过的文档数量
for ju in count_set:
    temp_l = ju.split()
    temp_s = set(temp_l)
    for w in temp_s:
        word_sens.setdefault(w, 0)
        word_sens[w] += 1
import pickle
pickle.dump(word_sens,open("/home/kesci/work/data/word_sens.pkl","wb"))

# 用前9亿5000万数据，统计title的点击次数C与曝光次数I

In [ ]:
import pandas as pd
import os
import pickle

split_data="/home/kesci/order_data"
data_list=os.listdir(split_data)
data_list.remove("19.csv")

all_title_I={} #曝光次数
all_title_C={} #点击次数

for d in data_list:
    print(d)
    d_index=d.split(".")[0]
    
    data_95=pd.read_csv(os.path.join(split_data,d),names=["query_id","query","title_id","title","label"])
    t_count=data_95["title"].value_counts().to_dict()
    t_click=data_95.groupby(["title"])["label"].sum().to_dict()
    for t in t_count:
        all_title_I.setdefault(t,0)
        all_title_I[t]+=t_count[t]
    for t in t_click:
        all_title_C.setdefault(t,0)
        all_title_C[t]+=t_click[t]

pickle.dump(all_title_I,open("/home/kesci/all_title_I.pkl","wb"))
pickle.dump(all_title_C,open("/home/kesci/all_title_C.pkl","wb"))

# 用最后5000万数据统计，统计title的点击次数C与曝光次数I

In [ ]:
all_title_I={}
all_title_C={}

data_95=pd.read_csv("/home/kesci/order_data/19.csv",names=["query_id","query","title_id","title","label"])
t_count=data_95["title"].value_counts().to_dict()
t_click=data_95.groupby(["title"])["label"].sum().to_dict()
for t in t_count:
    all_title_I.setdefault(t,0)
    all_title_I[t]+=t_count[t]
for t in t_click:
    all_title_C.setdefault(t,0)
    all_title_C[t]+=t_click[t]
    
pickle.dump(all_title_I,open("/home/kesci/title_I_19.pkl","wb"))
pickle.dump(all_title_C,open("/home/kesci/title_C_19.pkl","wb"))

# 用前9亿5000万数据统计，query和title里面每个词的曝光次数和点击次数

In [ ]:
import pickle

temp_q_dict = {}
sum_q_dict = {}
temp_t_dict = {}
sum_t_dict = {}
f=open("/home/kesci/input/bytedance/train_final.csv")
for myindex,line in enumerate(f,1):
    if myindex>950000000:
        break
    line = line.strip().split(",")
    query = set(line[1].split())
    title = set(line[3].split())
    label = int(line[4])
    for w in query:
        temp_q_dict.setdefault(w, 0)
        temp_q_dict[w] += label
        sum_q_dict.setdefault(w, 0)
        sum_q_dict[w] += 1
    for w in title:
        temp_t_dict.setdefault(w, 0)
        temp_t_dict[w] += label
        sum_t_dict.setdefault(w, 0)
        sum_t_dict[w] += 1
        
pickle.dump(temp_q_dict, open("temp_q_dict.pkl","wb"))
pickle.dump(sum_q_dict, open("sum_q_dict.pkl", "wb"))
pickle.dump(temp_t_dict, open("temp_t_dict.pkl", "wb"))
pickle.dump(sum_t_dict, open("sum_t_dict.pkl", "wb"))
f.close()

# 贝叶斯平滑title的ctr，以及query和title中词的ctr

In [ ]:
import numpy
import scipy.special as special
import numpy as np


class BayesianSmoothing(object):
    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta

    def sample(self, alpha, beta, num, imp_upperbound):
        sample = numpy.random.beta(alpha, beta, num)
        print(sample)
        I = []
        C = []
        for clk_rt in sample:
            imp = imp_upperbound
            clk = imp * clk_rt
            I.append(imp)
            C.append(clk)
        return I, C

    def update(self, imps, clks, iter_num, epsilon):
        for i in range(iter_num):
            new_alpha, new_beta = self.__fixed_point_iteration(imps, clks, self.alpha, self.beta)
            if abs(new_alpha - self.alpha) < epsilon and abs(new_beta - self.beta) < epsilon:
                break
            self.alpha = new_alpha
            self.beta = new_beta
            print("第"+str(i)+"轮 alpha: ",self.alpha,"beta: ",self.beta)

    def __fixed_point_iteration(self, imps, clks, alpha, beta):
        numerator_alpha = 0.0
        numerator_beta = 0.0
        denominator = 0.0

        for i in range(len(imps)):
            numerator_alpha += (special.digamma(clks[i] + alpha) - special.digamma(alpha))
            numerator_beta += (special.digamma(imps[i] - clks[i] + beta) - special.digamma(beta))
            denominator += (special.digamma(imps[i] + alpha + beta) - special.digamma(alpha + beta))

        return alpha * (numerator_alpha / denominator), beta * (numerator_beta / denominator)

def get_init_alpha_beta(old_ctrs):
    old_ctrs_mean=old_ctrs.mean()
    print("ctr_mean:",old_ctrs_mean)
    old_ctrs_var=old_ctrs.var()
    alpha = old_ctrs_mean/old_ctrs_var*(old_ctrs_mean*(1-old_ctrs_mean)-old_ctrs_var)
    beta = (1-old_ctrs_mean)/old_ctrs_var*(old_ctrs_mean*(1-old_ctrs_mean)-old_ctrs_var)
    return alpha,beta

In [ ]:
# 贝叶斯平滑title的ctr
import pickle
all_title_C=pickle.load(open("/home/kesci/all_title_C.pkl","rb"))
all_title_I=pickle.load(open("/home/kesci/all_title_I.pkl","rb"))
all_title_ctr={}
for t in all_title_I:
    all_title_ctr[t]=all_title_C[t]/all_title_I[t]
    
def smooth_ctr(old_ctrs,I_dict,C_dict,title_list,num_iters=100000,epsilon=0.0000000001):
    I=[]
    C=[]
    for t in title_list:
        I.append(I_dict[t])
        C.append(C_dict[t])
    init_alpha,init_beta=get_init_alpha_beta(np.array(list(old_ctrs.values()),dtype=np.float))
    print("初始化参数：",init_alpha,init_beta)
    bs = BayesianSmoothing(init_alpha,init_beta)
    bs.update(I,C,num_iters,epsilon)
    print("收敛参数：alpha: ",bs.alpha,"beta: ",bs.beta)

smooth_ctr(all_title_ctr,all_title_I,all_title_C,title_list=list(all_title_ctr.keys()),num_iters=200)
# 由于时间花费的比较长，所以后面的代码用我跑了151轮的平滑参数
# alpha=1.3912001443482402 beta=7.3260734359558795

In [ ]:
# 贝叶斯平滑title中词的ctr
import pickle
all_title_C=pickle.load(open("/home/kesci/work/temp_t_dict.pkl","rb"))
all_title_I=pickle.load(open("/home/kesci/work/sum_t_dict.pkl","rb"))
all_title_ctr={}
for t in all_title_I:
    all_title_ctr[t]=all_title_C[t]/all_title_I[t]
smooth_ctr(all_title_ctr,all_title_I,all_title_C,title_list=list(all_title_ctr.keys()),num_iters=3321)
# 由于时间花费的比较长，所以后面的代码用我跑了3321轮的平滑参数
# alpha=9.316810724554587 beta=46.161204596782255

In [ ]:
import pickle
countdict = pickle.load(open("./data/word_sens.pkl","rb"))
all_title_C=pickle.load(open("/home/kesci/work/temp_t_dict.pkl","rb"))
all_title_I=pickle.load(open("/home/kesci/work/sum_t_dict.pkl","rb"))
all_title_ctr={}
default_ctr=9.316810724554587/(9.316810724554587+46.161204596782255)
for t in countdict:
    if t in all_title_I:
        all_title_ctr[t]=(all_title_C[t]+9.316810724554587)/(all_title_I[t]+9.316810724554587+46.161204596782255)
    else:
        all_title_ctr[t]=default_ctr
pickle.dump(all_title_ctr,open("/home/kesci/title_word_ctr.pkl","wb"))#每个title里面词的ctr

In [ ]:
# 贝叶斯平滑query中词的ctr
import pickle
all_title_C=pickle.load(open("/home/kesci/work/temp_q_dict.pkl","rb"))
all_title_I=pickle.load(open("/home/kesci/work/sum_q_dict.pkl","rb"))
all_title_ctr={}
for t in all_title_I:
    all_title_ctr[t]=all_title_C[t]/all_title_I[t]
smooth_ctr(all_title_ctr,all_title_I,all_title_C,title_list=list(all_title_ctr.keys()),num_iters=4717)
# 由于时间花费的比较长，所以后面的代码用我跑了4717轮的平滑参数
# alpha=23.805871972380245 beta=120.9641867898008

import pickle
countdict=pickle.load(open("./data/word_sens.pkl","rb"))
all_title_C=pickle.load(open("/home/kesci/work/temp_q_dict.pkl","rb"))
all_title_I=pickle.load(open("/home/kesci/work/sum_q_dict.pkl","rb"))

all_title_ctr={}
default_ctr=23.805871972380245/(23.805871972380245+120.9641867898008)
for t in countdict:
    if t in all_title_I:
        all_title_ctr[t]=(all_title_C[t]+23.805871972380245)/(all_title_I[t]+23.805871972380245+120.9641867898008)
    else:
        all_title_ctr[t]=default_ctr
pickle.dump(all_title_ctr,open("/home/kesci/query_word_ctr.pkl","wb"))#每个query里面词的ctr

# 统计最后5000万数据和最终1亿测试集的每个query和title的每个词的tfidf值

In [ ]:
from collections import Counter
import math
import pickle

def tf(word, count , line_len):
    return count[word] / line_len


# 统计的是含有该单词的句子数
def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)


# len(count_list)是指句子的总数，n_containing(word, count_list)是指含有该单词的句子的总数，加1是为了防止分母为0
def idf(word, word_sens,ju_sum):
    return math.log(ju_sum / (1 + word_sens[word]))


# 将tf和idf相乘
def tfidf(word,count,word_sens,line_len,ju_sum):
    return tf(word,count,line_len) * idf(word,word_sens,ju_sum)
    
word_sens=pickle.load(open("./data/word_sens.pkl","rb"))
ju_sum=237412321
def get_tfidf(txt):
    temp_l = txt.split()
    res = " ".join([str(tfidf(word, Counter(temp_l), word_sens, len(temp_l), ju_sum)) for word in temp_l])
    return res

import pandas as pd
import pickle
data_list=["/home/kesci/order_data/19.csv"]

for file in data_list:
    print(file)
    num=file.split("/")[-1].split(".")[0]
    print(num)
    all_tfidf={}
    dataset = pd.read_csv(file,names=["query_id","query","title_id","title","label"],usecols=["query","title"])
    for txt in dataset["query"].unique():
        if txt not in all_tfidf:
            all_tfidf[txt]=get_tfidf(txt)
    
    for txt in dataset["title"].unique():
        if txt not in all_tfidf:
            all_tfidf[txt]=get_tfidf(txt)
    pickle.dump(all_tfidf,open("/home/kesci/all_txt_tfidf_"+num+".pkl","wb"))

data_list=["/home/kesci/test2.csv"]

for file in data_list:
    print(file)
    num=file.split("/")[-1].split(".")[0]
    print(num)
    all_tfidf={}
    dataset = pd.read_csv(file,names=["query_id","query","title_id","title"],usecols=["query","title"])
    for txt in dataset["query"].unique():
        if txt not in all_tfidf:
            all_tfidf[txt]=get_tfidf(txt)
    
    for txt in dataset["title"].unique():
        if txt not in all_tfidf:
            all_tfidf[txt]=get_tfidf(txt)
    pickle.dump(all_tfidf,open("/home/kesci/all_txt_tfidf_"+num+".pkl","wb"))

# 特征工程，对最后5000万和最终1亿测试集构造特征

In [ ]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from gensim.models import Word2Vec
import random
import Levenshtein
from scipy.spatial.distance import minkowski, correlation, cityblock, braycurtis, cosine, euclidean,canberra,chebyshev

In [ ]:
# 得到子序列在query和title中的开始位置以及结束位置
def get_startendpos(value, length):
	query_begin = 0
	query_end = 0
	title_begin = 0
	title_end = 0
	chushi_v = 0
	count = 0
	m, n = value.shape
	for i in range(1, m):
		for j in range(1, n):
			if int(value[i][j]) > chushi_v:
				if count == 0:
					query_begin = i
					title_begin = j
				count += 1
				chushi_v = int(value[i][j])
				if count == length:
					query_end = i
					title_end = j
					break
	return query_begin, query_end, title_begin, title_end

In [ ]:
#得到最大公共子序列的长度，最长公共子序列在query中的跨度，最长公共子序列在title中的跨度，最长子序列长度/title长度
def get_max_same_xulie(query, title): 
	q_list = query.split()
	query_len = len(q_list)
	t_list = title.split()
	title_len = len(t_list)
	my_value = np.zeros((query_len + 1, title_len + 1))
	for i in range(1, query_len + 1):
		for j in range(1, title_len + 1):
			if q_list[i - 1] == t_list[j - 1]:
				my_value[i][j] = my_value[i - 1][j - 1] + 1
			else:
				my_value[i][j] = max(my_value[i - 1][j], my_value[i][j - 1])
	max_value = my_value.max()
	query_begin, query_end, title_begin, title_end = get_startendpos(my_value, int(max_value))
	return max_value, (query_end - query_begin) / query_len,(title_end - title_begin) / title_len, max_value / title_len

In [2]:
# 计算了query与title的最大相同子串长度，起始位置，平均位置，相同子串总长度，相同密度比，
# 最大相同子串占query比，最大相同子串占title比
def get_son_str_feature(query, title):
	q_list = query.split()
	query_len = len(q_list)
	t_list = title.split()
	title_len = len(t_list)
	count1 = np.zeros((query_len + 1, title_len + 1))
	index = np.zeros((query_len + 1, title_len + 1))
	for i in range(1, query_len + 1):
		for j in range(1, title_len + 1):
			if q_list[i - 1] == t_list[j - 1]:
				count1[i][j] = count1[i - 1][j - 1] + 1
				index[i][j] = index[i - 1][j - 1] + j
			else:
				count1[i][j] = 0
				index[i][j] = 0
	max_count1 = count1.max()

	if max_count1 != 0:
		row = int(np.where(count1 == np.max(count1))[0][0])
		col = int(np.where(count1 == np.max(count1))[1][0])
		mean_pos = index[row][col] / (max_count1 * title_len)
		begin_loc = (col - max_count1 + 1) / title_len
		rows = np.where(count1 != 0.0)[0]
		cols = np.where(count1 != 0.0)[1]
		total_loc = 0
		for i in range(0, len(rows)):
			total_loc += index[rows[i]][cols[i]]
		density = total_loc / (query_len * title_len)
		rate_q_len = max_count1 / query_len
		rate_t_len = max_count1 / title_len
	else:
		begin_loc, mean_pos, total_loc, density, rate_q_len, rate_t_len = 0, 0, 0, 0, 0, 0
	return max_count1, begin_loc, mean_pos, total_loc, density, rate_q_len, rate_t_len

In [3]:
# 计算query和title中去重的相同词占所有单词的比例，
# query在title中重合的词的长度与title在query中重合的词的长度的和除以query词的长度与title词的长度的和，
# 相同词的长度占query长度比，
# 相同词的长度占title长度比，
# 每个相同词在query词中的平均位置，
# 每个相同词在title词中的平均位置，
# 去重的相同词占去重的query的长度比，
# 去重的相同词占去重的title的长度比，
# 两倍的去重的相同词的长度除以去重的title的词长度与去重的query的词长度的和，
# 每个去重的相同词在去重的query词中的平均位置，
# 每个去重的相同词在去重的title词中的平均位置，
# 去重的相同词长度与今昔词的长度的比例
def get_same_word_features(query, title):
	q_list = query.split()
	t_list = title.split()
	set_query = set(q_list)
	set_title = set(t_list)
	count_words = len(set_query.union(set_title))

	comwords = [word for word in t_list if word in q_list]
	comwords_set = set(comwords)
	unique_rate = len(comwords_set) / count_words

	same_word1 = [w for w in q_list if w in t_list]
	same_word2 = [w for w in t_list if w in q_list]
	same_len_rate = (len(same_word1) + len(same_word2)) / (len(q_list) + len(t_list))
	if len(comwords) > 0:
		com_index1 = len(comwords)
		same_word_q = com_index1 / len(q_list)
		same_word_t = com_index1 / len(t_list)

		for word in comwords_set:
			index_list = [i for i, x in enumerate(q_list) if x == word]
			com_index1 += sum(index_list)
		q_loc = com_index1 / (len(q_list) * len(comwords))
		com_index2 = len(comwords)
		for word in comwords_set:
			index_list = [i for i, x in enumerate(t_list) if x == word]
			com_index2 += sum(index_list)
		t_loc = com_index2 / (len(t_list) * len(comwords))

		same_w_set_q = len(comwords_set) / len(set_query)
		same_w_set_t = len(comwords_set) / len(set_title)
		word_set_rate = 2 * len(comwords_set) / (len(set_query) + len(set_title))

		com_set_query_index = len(comwords_set)
		for word in comwords_set:
			index_list = [i for i, x in enumerate(q_list) if x == word]
			if len(index_list) > 0:
				com_set_query_index += index_list[0]
		loc_set_q = com_set_query_index / (len(q_list) * len(comwords_set))
		com_set_title_index = len(comwords_set)
		for word in comwords_set:
			index_list = [i for i, x in enumerate(t_list) if x == word]
			if len(index_list) > 0:
				com_set_title_index += index_list[0]
		loc_set_t = com_set_title_index / (len(t_list) * len(comwords_set))
		set_rate = (len(comwords_set) / len(comwords))
	else:
		unique_rate, same_len_rate, same_word_q, same_word_t, q_loc, t_loc, same_w_set_q, same_w_set_t, word_set_rate, loc_set_q, loc_set_t, set_rate = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
	return unique_rate, same_len_rate, same_word_q, same_word_t, q_loc, t_loc, same_w_set_q, same_w_set_t, word_set_rate, loc_set_q, loc_set_t, set_rate

In [4]:
from fuzzywuzzy import fuzz
import random
# 因为是词别变成了数字编码，所以为了排除编码的影响，将每个编码映射成一个汉字，就是将每个词看成一个字，之后忽略
# 顺序进行模糊匹配
def Unicode():
    val=random.randint(0x4e00,0x9fbf)
    return chr(val)
a=[]
Chinese=[]
for i in range(2000):
    a.append(Unicode())
a=set(a)#随机生成的汉字去重
for word in a:#转成list
    Chinese.append(word)

def get_fuzz_ratio1(df,model,reference):
    #映射为汉字
    query=df[model]
    title=df[reference]
    terms_model= query.split()
    terms_reference= title.split()
    grams_model = set(terms_model)
    grams_reference = set(terms_reference)
    Union=grams_model|grams_reference#讲两个set合并
    query_chinese=[]
    title_chinese=[]
    Dict={}
    count=0
    for num in Union:
        Dict[num]=Chinese[count]
        count+=1
    for num1 in terms_model:
        query_chinese.append(Dict[num1])
    for num2 in terms_reference:
        title_chinese.append(Dict[num2])
    str_query=' '
    str_query=str_query.join(query_chinese)
    str_title=' '
    str_title=str_title.join(title_chinese)
    token_sort_value = fuzz.token_sort_ratio(str_query, str_title)
    return token_sort_value

# 去重子集匹配
def get_fuzz_ratio2(df,model,reference):
    #映射为汉字
    query=df[model]
    title=df[reference]
    terms_model= query.split()
    terms_reference= title.split()
    grams_model = set(terms_model)
    grams_reference = set(terms_reference)
    Union=grams_model|grams_reference#讲两个set合并
    query_chinese=[]
    title_chinese=[]
    Dict={}
    count=0
    for num in Union:
        Dict[num]=Chinese[count]
        count+=1
    for num1 in terms_model:
        query_chinese.append(Dict[num1])
    for num2 in terms_reference:
        title_chinese.append(Dict[num2])
    str_query=' '
    str_query=str_query.join(query_chinese)
    str_title=' '
    str_title=str_title.join(title_chinese)
    token_set_value = fuzz.token_set_ratio(str_query, str_title)
    return token_set_value

In [ ]:
# 结合tfidf与余弦相似度，将词频换成tfidf值，计算query和title的余弦相似度
def cos_dist(vec1, vec2):
    """
    :param vec1: 向量1
    :param vec2: 向量2
    :return: 返回两个向量的余弦相似度
    """
    dist1 = float(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))
    return dist1

def get_tfidf_scores(df, s1, s2):
    """
    :param s1: 句子1
    :param s2: 句子2
    :return: 返回句子的余弦相似度
    """
    # 分词

    list_word1 = df[s1].split()
    list_word2 = df[s2].split()
    tfidf_score1 = {i[0]: float(i[1]) for i in zip(list_word1, all_tfidf[df[s1]].split())}
    tfidf_score2 = {i[0]: float(i[1]) for i in zip(list_word2, all_tfidf[df[s2]].split())}

    # 列出所有的词,取并集
    key_word = list(set(list_word1 + list_word2))

    # 给定形状和类型的用0填充的矩阵存储向量
    word_vector1 = np.zeros(len(key_word))
    word_vector2 = np.zeros(len(key_word))

    # 计算词频
    # 依次确定向量的每个位置的值
    for i in range(len(key_word)):
        # 遍历key_word中每个词在句子中的出现次数
        for j in range(len(list_word1)):
            if key_word[i] == list_word1[j]:
                word_vector1[i] += tfidf_score1[key_word[i]]
        for k in range(len(list_word2)):
            if key_word[i] == list_word2[k]:
                word_vector2[i] += tfidf_score2[key_word[i]]
    return cos_dist(word_vector1, word_vector2)

In [ ]:
# 提取query与title中互不重复的独立词
def extr_words(str_query, str_title):
    list_query = str_query.split()
    list_title = str_title.split()

    query_word = [word for word in list_query if word not in list_title]
    title_word = [word for word in list_title if word not in list_query]

    return query_word, title_word

# 提取query和title中所有词，加载词向量，将query和title所有词的词向量分别相加后求平均，
# 得到的query平均词向量和title平均词向量进行多种距离函数计算其平均词向量相似度
def get_sim(str_query, str_title):
    list_query = str_query.split()
    list_title = str_title.split()
    query = []
    for word in list_query:
        if word_vectors.__contains__(word):
            query.append(word_vectors.__getitem__(word))
    query = np.array(query)
    if len(query) != 0:
        q_vec = query.sum(axis=0) / query.shape[0]
    else:
        q_vec = np.zeros((128,), dtype=np.float32)

    title = []
    for word in list_title:
        if word_vectors.__contains__(word):
            title.append(word_vectors.__getitem__(word))
    title = np.array(title)
    if len(title) != 0:
        t_vec = title.sum(axis=0) / title.shape[0]
    else:
        t_vec = np.zeros((128,), dtype=np.float32)
    dot_prod = np.sum(np.multiply(q_vec, t_vec))  
    braycurtis_val = braycurtis(q_vec, t_vec)
    canberra_val = canberra(q_vec, t_vec)
    euclidean_val = euclidean(q_vec, t_vec)
    minkowski_val = minkowski(q_vec, t_vec, p=3)
    cityblock_val = cityblock(q_vec, t_vec)
    correlation_val = correlation(q_vec, t_vec)
    cosine_val = cosine(q_vec, t_vec)
    chebyshev_val = chebyshev(q_vec, t_vec)
    if cosine_val == 'nan' or cosine_val == 'inf':
        cosine_val = 2
    if braycurtis_val == 'nan' or braycurtis_val == 'inf':
        braycurtis_val = 2
    if correlation_val == 'nan' or correlation_val == 'inf':
        correlation_val = 1

    return dot_prod, braycurtis_val, canberra_val, euclidean_val, minkowski_val, cityblock_val, correlation_val, cosine_val, chebyshev_val

In [ ]:
# 提取query和title中互不相同的独立词，
# 加载词向量，将query和title独立词的词向量分别相加后求平均，
# 得到的query平均独立词向量和title平均独立词向量进行多种距离函数计算其平均独立词向量距离
def get_dis(str_query, str_title):
    list_query, list_title = extr_words(str_query, str_title)
    query = []
    for word in list_query:
        if word_vectors.__contains__(word):
            query.append(word_vectors.__getitem__(word))
    query = np.array(query)
    if len(query) != 0:
        q_vec = query.sum(axis=0) / query.shape[0]
    else:
        q_vec = np.zeros((128,), dtype=np.float32)
    title = []
    for word in list_title:
        if word_vectors.__contains__(word):
            title.append(word_vectors.__getitem__(word))
    title = np.array(title)
    if len(title) != 0:
        t_vec = title.sum(axis=0) / title.shape[0]
    else:
        t_vec = np.zeros((128,), dtype=np.float32)
    dis_dot_prod = np.sum(np.multiply(q_vec, t_vec))  
    dis_braycurtis_val = braycurtis(q_vec, t_vec)
    dis_euclidean_val = euclidean(q_vec, t_vec)
    dis_minkowski_val = minkowski(q_vec, t_vec, p=3)
    dis_canberra_val = canberra(q_vec, t_vec)
    dis_cityblock_val = cityblock(q_vec, t_vec)
    dis_correlation_val = correlation(q_vec, t_vec)
    dis_cosine_val = cosine(q_vec, t_vec)
    dis_chebyshev_val = chebyshev(q_vec, t_vec)

    if dis_cosine_val == 'nan' or dis_cosine_val == 'inf':
        dis_cosine_val = 2
    if dis_braycurtis_val == 'nan' or dis_braycurtis_val == 'inf':
        dis_braycurtis_val = 2
    if dis_correlation_val == 'nan' or dis_correlation_val == 'inf':
        dis_correlation_val = 1
    return dis_dot_prod, dis_braycurtis_val, dis_euclidean_val, dis_minkowski_val, dis_canberra_val, dis_cityblock_val, dis_correlation_val, dis_cosine_val, dis_chebyshev_val

In [ ]:
# 因为是词别变成了数字编码，所以为了排除编码的影响，将每个编码映射成一个汉字，就是将每个词看成一个字。
# 计算莱文斯坦比。
# 计算公式  r = (sum - ldist) / sum, 其中sum是指str1 和 str2 字串的长度总和，ldist是 类编辑距离
def cal_ratio(df,model,reference):
    #映射为汉字
    query=df[model]
    title=df[reference]
    terms_model= query.split()
    terms_reference= title.split()
    grams_model = set(terms_model)
    grams_reference = set(terms_reference)
    Union=grams_model|grams_reference#讲两个set合并
    query_chinese=[]
    title_chinese=[]
    Dict={}
    count=0
    for num in Union:
        Dict[num]=Chinese[count]
        count+=1
    for num1 in terms_model:
        query_chinese.append(Dict[num1])
    for num2 in terms_reference:
        title_chinese.append(Dict[num2])
    str_query=''
    str_query=str_query.join(query_chinese)
    str_title=''
    str_title=str_title.join(title_chinese)
    ratio=Levenshtein.ratio(str_query,str_title)
    return ratio

# 计算query和title的jaro距离
def cal_chinese_jaro(df,model,reference):
    #映射为汉字
    query=df[model]
    title=df[reference]
    terms_model= query.split()
    terms_reference= title.split()
    grams_model = set(terms_model)
    grams_reference = set(terms_reference)
    Union=grams_model|grams_reference#讲两个set合并
    query_chinese=[]
    title_chinese=[]
    Dict={}
    count=0
    for num in Union:
        Dict[num]=Chinese[count]
        count+=1
    for num1 in terms_model:
        query_chinese.append(Dict[num1])
    for num2 in terms_reference:
        title_chinese.append(Dict[num2])
    str_query=''
    str_query=str_query.join(query_chinese)
    str_title=''
    str_title=str_title.join(title_chinese)
    ratio=Levenshtein.jaro(str_query,str_title)
    return ratio

# 计算query和title的Jaro–Winkler距离
def cal_chinese_jaro_winkler(df,model,reference):
    #映射为汉字
    query=df[model]
    title=df[reference]
    terms_model= query.split()
    terms_reference= title.split()
    grams_model = set(terms_model)
    grams_reference = set(terms_reference)
    Union=grams_model|grams_reference#讲两个set合并
    query_chinese=[]
    title_chinese=[]
    Dict={}
    count=0
    for num in Union:
        Dict[num]=Chinese[count]
        count+=1
    for num1 in terms_model:
        query_chinese.append(Dict[num1])
    for num2 in terms_reference:
        title_chinese.append(Dict[num2])
    str_query=''
    str_query=str_query.join(query_chinese)
    str_title=''
    str_title=str_title.join(title_chinese)
    ratio=Levenshtein.jaro_winkler(str_query,str_title)
    return ratio

In [5]:
# 计算query和title的BM25相关性
import math
def calBM25(data1,col1,col2):
    querytemp=data1[col1].split()
    titletemp=data1[col2].split()
    dict_temp={}
    for word in querytemp:
        if word in dict_temp:
            dict_temp[word]+=1
        else:
            dict_temp[word]=1
    sumscore=0
    sl=len(titletemp)
    for word in titletemp: 
        if word in countdict and word in dict_temp:
            fi=dict_temp[word]
            sumscore+=math.log(237412321 / (1 + countdict[word]))*fi*(2+1)/(fi+2*(1-0.75+0.75*sl/10))
    return sumscore

In [6]:
#计算query和title的2-gram和3-gram，相同数
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import os

class StringDistance:

    def distance(self, s0, s1):
        raise NotImplementedError()


class NormalizedStringDistance(StringDistance):

    def distance(self, s0, s1):
        raise NotImplementedError()


class MetricStringDistance(StringDistance):

    def distance(self, s0, s1):
        raise NotImplementedError()

class NGram(NormalizedStringDistance):

    def __init__(self, n=2):
        self.n = n

    def distance(self, s0, s1):
        if s0 is None:
            raise TypeError("Argument s0 is NoneType.")
        if s1 is None:
            raise TypeError("Argument s1 is NoneType.")
        if s0 == s1:
            return 0.0

        special = '\n'
        sl = len(s0)
        tl = len(s1)

        if sl == 0 or tl == 0:
            return 1.0

        cost = 0
        if sl < self.n or tl < self.n:
            for i in range(min(sl, tl)):
                if s0[i] == s1[i]:
                    cost += 1
            return 1.0 * cost / max(sl, tl)

        sa = [''] * (sl + self.n - 1)

        for i in range(len(sa)):
            if i < self.n - 1:
                sa[i] = special
            else:
                sa[i] = s0[i - self.n + 1]

        p = [0.0] * (sl + 1)
        d = [0.0] * (sl + 1)
        t_j = [''] * self.n
        for i in range(sl + 1):
            p[i] = 1.0 * i

        for j in range(1, tl + 1):
            if j < self.n:
                for ti in range(self.n - j):
                    t_j[ti] = special
                for ti in range(self.n - j, self.n):
                    t_j[ti] = s1[ti - (self.n - j)]
            else:
                t_j = s1[j - self.n:j]

            d[0] = 1.0 * j
            for i in range(sl + 1):
                cost = 0
                tn = self.n
                for ni in range(self.n):
                    if sa[i - 1 + ni] != t_j[ni]:
                        cost += 1
                    elif sa[i - 1 + ni] == special:
                        tn -= 1
                ec = cost / tn
                d[i] = min(d[i - 1] + 1, p[i] + 1, p[i - 1] + ec)
            p, d = d, p

        return p[sl] / max(tl, sl)
twogram = NGram(2)
trigram = NGram(3)

def two_gram(df,col1,col2):
    Query=df[col1].split()
    Title=df[col2].split()
    Two_gram=twogram.distance(Query,Title)
    return Two_gram

def tri_gram(df,col1,col2):
    Query=df[col1].split()
    Title=df[col2].split()
    Tri_gram=trigram.distance(Query,Title)
    return Tri_gram

In [7]:
#计算query和title的编辑距离
def edit_distance(df, w1, w2):
    word1 = df[w1].split()
    word2 = df[w2].split()
    len1 = len(word1)
    len2 = len(word2)
    dp = np.zeros((len1 + 1, len2 + 1))
    for i in range(len1 + 1):
        dp[i][0] = i
    for j in range(len2 + 1):
        dp[0][j] = j

    for i in range(1, len1 + 1):
        for j in range(1, len2 + 1):
            delta = 0 if word1[i - 1] == word2[j - 1] else 1
            dp[i][j] = min(dp[i - 1][j - 1] + delta, min(dp[i - 1][j] + 1, dp[i][j - 1] + 1))
    return dp[len1][len2]

In [8]:
# 得到query中tfidf最大的那个词的ctr
def get_best_query_word_ctr(txt):
    all_w=txt.split()
    tfidf_score1 = {i[0]: float(i[1]) for i in zip(all_w, all_tfidf[txt].split())}
    p_d=sorted(tfidf_score1.items(),key=lambda item:item[1],reverse=True)
    return query_word_ctr[p_d[0][0]]
  
# 得到title中tfidf最大的那个词的ctr 
def get_best_title_word_ctr(txt):
    all_w=txt.split()
    tfidf_score1 = {i[0]: float(i[1]) for i in zip(all_w, all_tfidf[txt].split())}
    p_d=sorted(tfidf_score1.items(),key=lambda item:item[1],reverse=True)
    return title_word_ctr[p_d[0][0]]

In [9]:
# 得到query的每个词的tfidf值和每个词的ctr的加权和。
def get_sum_query_word_ctr(txt):
    all_w=txt.split()
    tfidf_score=np.array([float(i) for i in all_tfidf[txt].split()])
    ctr_score=np.array([query_word_ctr[i] for i in all_w])
    return np.dot(tfidf_score,ctr_score)

# 得到title的每个词的tfidf值和每个词的ctr的加权和。
def get_sum_title_word_ctr(txt):
    all_w=txt.split()
    tfidf_score=np.array([float(i) for i in all_tfidf[txt].split()])
    ctr_score=np.array([title_word_ctr[i] for i in all_w])
    return np.dot(tfidf_score,ctr_score)

In [10]:
data = pd.read_csv("/home/kesci/order_data/19.csv",names=["query_id","query","title_id","title","label"])


data["long_same_lie"],data["long_same_lie_q"],data["long_same_lie_t"],data["long_same_t_rate"]=zip(*data.apply(lambda line: get_max_same_xulie(line["query"], line["title"]),axis=1))
data[["long_same_lie","long_same_lie_q","long_same_lie_t","long_same_t_rate"]].to_csv(
    "/home/kesci/work/features/train/long_same_lie.csv",index=False)


data["long_same_max_count1"], data["long_same_local_begin"], data["long_same_local_mean"],data["long_same_total_loc"],\
data["long_same_density"], data["long_same_rate_q_len"], data["long_same_rate_t_len"]= zip(
    *data.apply(lambda line: get_son_str_feature(line["query"], line["title"]), axis=1))
data[["long_same_max_count1","long_same_local_begin","long_same_local_mean","long_same_total_loc",
"long_same_density","long_same_rate_q_len","long_same_rate_t_len"]].to_csv(
    "/home/kesci/work/features/train/long_same_str.csv",index=False)


data["unique_rate"],data["same_len_rate"],data["same_word_q"],\
data["same_word_t"],data["q_loc"],data["t_loc"],data["same_w_set_q"],data["same_w_set_t"],data["word_set_rate"],\
data["loc_set_q"], data["loc_set_t"], data["set_rate"]= zip(
    *data.apply(lambda line: get_same_word_features(line["query"], line["title"]), axis=1))
data[["unique_rate", "same_len_rate", "same_word_q", "same_word_t", "q_loc", "t_loc", "same_w_set_q", "same_w_set_t", "word_set_rate", "loc_set_q", "loc_set_t", "set_rate"]].to_csv(
    "/home/kesci/work/features/train/same_w_q_t.csv",index=False)


data['fuzz_1']=data.apply(get_fuzz_ratio1,axis=1,args=('query','title'))
data['fuzz_2']=data.apply(get_fuzz_ratio2,axis=1,args=('query','title'))
data[["fuzz_1","fuzz_2"]].to_csv(
    "/home/kesci/work/features/train/fuzz.csv",index=False)


import pickle
all_tfidf=pickle.load(open("/home/kesci/all_txt_tfidf_19.pkl","rb"))
data["cos_tfidf_distance"] = data.apply(get_tfidf_scores,axis = 1,args = ('query','title'))
data[["cos_tfidf_distance"]].to_csv("/home/kesci/work/features/train/cos_tfidf_distance.csv",index=False)


model = Word2Vec.load("/home/kesci/word2vec.model")
word_vectors = model.wv
del model
data["sim_1"], data["sim_2"], data["sim_3"], data["sim_4"], data["sim_5"],data["sim_6"], \
data["sim_7"],data["sim_8"],data["sim_9"]= zip(
    *data.apply(lambda line: get_sim(line["query"], line["title"]), axis=1))
data[["sim_1","sim_2","sim_3","sim_4","sim_5","sim_6","sim_7","sim_8","sim_9"]].fillna(0).to_csv(
    "/home/kesci/work/features/train/sim_feature.csv",
index=False)


data["dis_1"], data["dis_2"], data["dis_3"], data["dis_4"],data["dis_5"], data["dis_6"], data["dis_7"],\
data["dis_8"],data["dis_9"] = zip(
    *data.apply(lambda line: get_dis(line["query"], line["title"]), axis=1))
data[["dis_1","dis_2","dis_3","dis_4","dis_5","dis_6","dis_7","dis_8","dis_9"]].fillna(0).to_csv(
    "/home/kesci/work/features/train/dis_feature.csv",index=False)


data['Chinese_ratio']=data.apply(cal_ratio,axis=1,args=('query','title'))
data['jaro']=data.apply(cal_chinese_jaro,axis=1,args=('query','title'))
data['jaro_winkler']=data.apply(cal_chinese_jaro_winkler,axis=1,args=('query','title'))
data[["Chinese_ratio","jaro","jaro_winkler"]].to_csv("/home/kesci/work/features/train/Levenshtein.csv",index=False)


import pickle
countdict=pickle.load(open("./data/word_sens.pkl","rb"))
data['BM25'] = data.apply(calBM25,axis=1,args=('query','title'))
data[["BM25"]].to_csv("/home/kesci/work/features/train/bm25.csv",index=False)


data['two_gram']=data.apply(two_gram,axis=1,args=('query','title'))
data['tri_gram']=data.apply(tri_gram,axis=1,args=('query','title'))
data[['two_gram','tri_gram']].to_csv("/home/kesci/work/features/train/gram.csv",index=False)


data["edit_distance"] = data.apply(edit_distance, axis=1, args=('query', 'title'))
data[["edit_distance"]].to_csv("/home/kesci/work/features/train/edit_distance.csv",index=False)


# 计算query和title的长度
data["query_len"]=data["query"].apply(lambda x:len(x.split()))
data["title_len"]=data["title"].apply(lambda x:len(x.split()))
data[["query_len","title_len"]].to_csv("/home/kesci/work/features/train/query_and_title_len.csv",index=False)


data["best_query_word_ctr"] = data["query"].apply(get_best_query_word_ctr)
data["best_title_word_ctr"] = data["title"].apply(get_best_title_word_ctr)
data["sum_query_word_ctr"] = data["query"].apply(get_sum_query_word_ctr)
data["sum_title_word_ctr"] = data["title"].apply(get_sum_title_word_ctr)
data[["best_query_word_ctr","best_title_word_ctr","sum_query_word_ctr","sum_title_word_ctr"]].to_csv(
    "/home/kesci/work/features/train/query_title_word_ctr.csv",
index=False)


# 利用最后5000万数据进行，5折交叉统计title的ctr
data["5_ctr"]=None
alpha=1.3912001443482402
beta=7.3260734359558795
ctr_5=[]
for i in range(5):
    if i==0:
        data_95=data.loc[10000000:]
    elif i==4:
        data_95=data.loc[:4*10000000-1]
    else:
        data_95=pd.concat([data.loc[:i*10000000-1],data.loc[(i+1)*10000000:]],axis=0)
    all_title_I={}
    all_title_C={}
    t_count=data_95["title"].value_counts().to_dict()
    t_click=data_95.groupby(["title"])["label"].sum().to_dict()
    for t in t_count:
        all_title_I[t]=t_count[t]
    for t in t_click:
        all_title_C[t]=t_click[t]
    def get_5_ctr(title):
        if title in all_title_I:
            return (all_title_C[title]+alpha)/(all_title_I[title]+alpha+beta)
        else:
            return alpha/(alpha+beta)
    ctr_5.extend(list(data.loc[i*10000000:(i+1)*10000000-1]["title"].apply(get_5_ctr)))
data["5_ctr"]=ctr_5
data[["5_ctr"]].to_csv("/home/kesci/work/features/train/ctr_5.csv",index=False)

In [ ]:
data = pd.read_csv("/home/kesci/test2.csv",names=["query_id","query","title_id","title"])


data["long_same_lie"],data["long_same_lie_q"],data["long_same_lie_t"],data["long_same_t_rate"]=zip(*data.apply(lambda line: get_max_same_xulie(line["query"], line["title"]),axis=1))
data[["long_same_lie","long_same_lie_q","long_same_lie_t","long_same_t_rate"]].to_csv(
    "/home/kesci/work/features/test2/long_same_lie.csv",index=False)


data["long_same_max_count1"], data["long_same_local_begin"], data["long_same_local_mean"],data["long_same_total_loc"],\
data["long_same_density"], data["long_same_rate_q_len"], data["long_same_rate_t_len"]= zip(
    *data.apply(lambda line: get_son_str_feature(line["query"], line["title"]), axis=1))
data[["long_same_max_count1","long_same_local_begin","long_same_local_mean","long_same_total_loc",
"long_same_density","long_same_rate_q_len","long_same_rate_t_len"]].to_csv(
    "/home/kesci/work/features/test2/long_same_str.csv",index=False)


data["unique_rate"],data["same_len_rate"],data["same_word_q"],\
data["same_word_t"],data["q_loc"],data["t_loc"],data["same_w_set_q"],data["same_w_set_t"],data["word_set_rate"],\
data["loc_set_q"], data["loc_set_t"], data["set_rate"]= zip(
    *data.apply(lambda line: get_same_word_features(line["query"], line["title"]), axis=1))
data[["unique_rate", "same_len_rate", "same_word_q", "same_word_t", "q_loc", "t_loc", "same_w_set_q", "same_w_set_t", "word_set_rate", "loc_set_q", "loc_set_t", "set_rate"]].to_csv(
    "/home/kesci/work/features/test2/same_w_q_t.csv",index=False)


data['fuzz_1']=data.apply(get_fuzz_ratio1,axis=1,args=('query','title'))
data['fuzz_2']=data.apply(get_fuzz_ratio2,axis=1,args=('query','title'))
data[["fuzz_1","fuzz_2"]].to_csv(
    "/home/kesci/work/features/test2/fuzz.csv",index=False)


import pickle
all_tfidf=pickle.load(open("/home/kesci/all_txt_tfidf_test2.pkl","rb"))
data["cos_tfidf_distance"] = data.apply(get_tfidf_scores,axis = 1,args = ('query','title'))
data[["cos_tfidf_distance"]].to_csv("/home/kesci/work/features/test2/cos_tfidf_distance.csv",index=False)


model = Word2Vec.load("/home/kesci/word2vec.model")
word_vectors = model.wv
del model
data["sim_1"], data["sim_2"], data["sim_3"], data["sim_4"], data["sim_5"],data["sim_6"], \
data["sim_7"],data["sim_8"],data["sim_9"]= zip(
    *data.apply(lambda line: get_sim(line["query"], line["title"]), axis=1))
data[["sim_1","sim_2","sim_3","sim_4","sim_5","sim_6","sim_7","sim_8","sim_9"]].fillna(0).to_csv(
    "/home/kesci/work/features/test2/sim_feature.csv",
index=False)


data["dis_1"], data["dis_2"], data["dis_3"], data["dis_4"],data["dis_5"], data["dis_6"], data["dis_7"],\
data["dis_8"],data["dis_9"] = zip(
    *data.apply(lambda line: get_dis(line["query"], line["title"]), axis=1))
data[["dis_1","dis_2","dis_3","dis_4","dis_5","dis_6","dis_7","dis_8","dis_9"]].fillna(0).to_csv(
    "/home/kesci/work/features/test2/dis_feature.csv",index=False)


data['Chinese_ratio']=data.apply(cal_ratio,axis=1,args=('query','title'))
data['jaro']=data.apply(cal_chinese_jaro,axis=1,args=('query','title'))
data['jaro_winkler']=data.apply(cal_chinese_jaro_winkler,axis=1,args=('query','title'))
data[["Chinese_ratio","jaro","jaro_winkler"]].to_csv("/home/kesci/work/features/test2/Levenshtein.csv",index=False)


import pickle
countdict=pickle.load(open("./data/word_sens.pkl","rb"))
data['BM25'] = data.apply(calBM25,axis=1,args=('query','title'))
data[["BM25"]].to_csv("/home/kesci/work/features/test2/bm25.csv",index=False)


data['two_gram']=data.apply(two_gram,axis=1,args=('query','title'))
data['tri_gram']=data.apply(tri_gram,axis=1,args=('query','title'))
data[['two_gram','tri_gram']].to_csv("/home/kesci/work/features/test2/gram.csv",index=False)


data["edit_distance"] = data.apply(edit_distance, axis=1, args=('query', 'title'))
data[["edit_distance"]].to_csv("/home/kesci/work/features/test2/edit_distance.csv",index=False)


# 计算query和title的长度
data["query_len"]=data["query"].apply(lambda x:len(x.split()))
data["title_len"]=data["title"].apply(lambda x:len(x.split()))
data[["query_len","title_len"]].to_csv("/home/kesci/work/features/test2/query_and_title_len.csv",index=False)


data["best_query_word_ctr"] = data["query"].apply(get_best_query_word_ctr)
data["best_title_word_ctr"] = data["title"].apply(get_best_title_word_ctr)
data["sum_query_word_ctr"] = data["query"].apply(get_sum_query_word_ctr)
data["sum_title_word_ctr"] = data["title"].apply(get_sum_title_word_ctr)
data[["best_query_word_ctr","best_title_word_ctr","sum_query_word_ctr","sum_title_word_ctr"]].to_csv(
    "/home/kesci/work/features/test2/query_title_word_ctr.csv",
index=False)

# 多维特征训练lgb模型，含有部分特征构造(因为构造较快，所以不需要提前做好存储)

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import os
import gc
from sklearn.model_selection import train_test_split

In [ ]:
data_5 = pd.read_csv("/home/kesci/order_data/19.csv",names=["query_id","query","title_id","title","label"])

In [ ]:
# 每个query对应不同的title数量
# 每个title对应不同的query数量
data_5["query_title_count"] = data_5.groupby("query")["title"].transform("nunique")
data_5["title_query_count"] = data_5.groupby("title")["query"].transform("nunique")

In [ ]:
print(data_5["query_title_count"].mean())
print(data_5["title_query_count"].mean())
del data_5["query"],data_5["title"]
gc.collect()

In [ ]:
# 读取特征目录中的所有特征文件拼接到原数据上
feature_path="/home/kesci/work/features/train"
sl=os.listdir(feature_path)
sl.remove('ctr_5.csv')

for i in sl:
    temp_data=pd.read_csv(os.path.join(feature_path,i))
    print(len(temp_data),i)
    data_5 = pd.concat([data_5,temp_data],axis=1)
del temp_data
gc.collect()


# query和title长度的差值
data_5["dif_len"] = data_5["query_len"] - data_5["title_len"] 
# query和title长度的比值
data_5["ql_dev_tl"] = data_5["query_len"] / data_5["title_len"] 
# query组内的title长度均值
data_5["query_title_len_mean"] = data_5.groupby("query_id")["title_len"].transform("mean") 
# query的长度与query组内的title长度均值差值
data_5["q_dif_len_mean"] = data_5["query_len"] - data_5["query_title_len_mean"]
# title的长度与query组内的title长度均值差值
data_5["t_dif_len_mean"] = data_5["title_len"] - data_5["query_title_len_mean"]
# query的长度与query组内的title长度均值比值
data_5["q_dev_len_mean"] = data_5["query_len"] / data_5["query_title_len_mean"]
# title的长度与query组内的title长度均值比值
data_5["t_dev_len_mean"] = data_5["title_len"] / data_5["query_title_len_mean"]

In [ ]:
# sort_features要在query组内进行排序的特征
sort_features=["BM25","title_query_count","cos_tfidf_distance","sim_1","t_dev_len_mean","dis_1",
"t_loc","dis_8","t_dif_len_mean","loc_set_q","long_same_lie_t",
"loc_set_t","set_rate","long_same_total_loc","dis_7","long_same_density","q_loc",
"dis_2","same_word_t","sim_2","dis_9","sim_8","dis_5","sim_9","dis_6",
"title_len","dis_4",
"same_len_rate",
"sim_6","sim_7","tri_gram","sim_3","sim_5","jaro_winkler","long_same_rate_t_len",
"long_same_local_mean",
"dis_3","long_same_local_begin","two_gram","sim_4","ql_dev_tl","long_same_lie_q",
"long_same_rate_q_len"]

for fea in sort_features:
    print(fea)
    # 对特征进行组内排序
    data_5[fea+'_sort']=data_5[fea].groupby(data_5['query_id']).rank(ascending=0,method='dense')
    
del data_5["query_id"],data_5["title_id"]
gc.collect()

# 为了节省内存，将int64转化为int32
for i in data_5.columns:
    if "_sort" in i:
        print(i)
        data_5[i]=data_5[i].astype("int32")

# 为了节省内存，将float64转化为float32
for i in data_5.dtypes.index:
    if data_5.dtypes[i]=="float64":
        print(i)
        data_5[i]=data_5[i].astype("float32")
        
        
train_colums=list(data_5.columns)
train_colums.remove("label")
train_x=data_5[train_colums[0]].values
train_x=train_x.reshape([len(train_x),1])
for lie in train_colums[1:]:
    temp_data=data_5[lie].values
    temp_data=temp_data.reshape([len(temp_data),1])
    train_x=np.c_[train_x,temp_data]
del temp_data
gc.collect()

train_y=data_5["label"].values
import gc
del data_5
gc.collect()
X_train = train_x[:-50000]
y_train = train_y[:-50000]
X_test = train_x[-50000:]
y_test = train_y[-50000:]
import gc
del train_x
del train_y
gc.collect()
lgb_train = lgb.Dataset(X_train,y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)


params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {"auc",'binary_logloss'}, 
    'num_leaves': 76,
    'max_depth': -1,
     'subsample_freq':1,
     'feature_fraction':0.8,
    'learning_rate': 0.03,
    'colsample_bytree':0.8,
    'subsample':0.8,
     'reg_alpha':0.0,
     'reg_lambda':1,
    'min_child_weight':50
}
print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=5000,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)
gbm.save_model('./model_file/gbm_good_model10.txt')

# 生成第一个lgb模型的最终测试集的预测结果

In [ ]:
data_test = pd.read_csv("/home/kesci/test2.csv",names=["query_id","query","title_id","title"])
feature_path="/home/kesci/work/features/test2"
sl=os.listdir(feature_path)
for i in sl:
    temp_data=pd.read_csv(os.path.join(feature_path,i))
    print(len(temp_data),i)
    for i in temp_data.dtypes.index:
        if temp_data.dtypes[i]=="float64":
            print(i)
            temp_data[i]=temp_data[i].astype("float32")
    data_test = pd.concat([data_test,temp_data],axis=1)
    

data_test["dif_len"] = data_test["query_len"] - data_test["title_len"]
data_test["ql_dev_tl"] = data_test["query_len"] / data_test["title_len"]
data_test["query_title_len_mean"] = data_test.groupby("query_id")["title_len"].transform("mean")
data_test["q_dif_len_mean"] = data_test["query_len"] - data_test["query_title_len_mean"]
data_test["t_dif_len_mean"] = data_test["title_len"] - data_test["query_title_len_mean"]
data_test["q_dev_len_mean"] = data_test["query_len"] / data_test["query_title_len_mean"]
data_test["t_dev_len_mean"] = data_test["title_len"] / data_test["query_title_len_mean"]
data_test["query_title_count"] = data_test.groupby("query")["title"].transform("nunique")
data_test["title_query_count"] = data_test.groupby("title")["query"].transform("nunique")

In [ ]:
for fea in sort_features:
    print(fea)
    data_test[fea+'_sort']=data_test[fea].groupby(data_test['query_id']).rank(ascending=0,method='dense')
    
for i in data_test.columns:
    if "_sort" in i:
        print(i)
        data_test[i]=data_test[i].astype("int32")

for i in data_test.dtypes.index:
    if data_test.dtypes[i]=="float64":
        print(i)
        data_test[i]=data_test[i].astype("float32")

gbm=lgb.Booster(model_file='./model_file/gbm_good_model10.txt')


#为节省内存，每500万转换成一次numpy矩阵
preds=np.array([])
for i in range(0,len(data_test),5000000):
    print(i)
    temp_x=data_test.loc[i:i+4999999][train_colums].values
    print(len(temp_x))
    temp_preds = gbm.predict(temp_x, num_iteration=gbm.best_iteration)  
    preds=np.r_[preds,temp_preds]

test_pred=pd.DataFrame(preds)
test_pred.columns=["prediction"]
test_pred["query_id"]=list(data_test["query_id"])
test_pred["title_id"]=list(data_test["title_id"])
test_pred[["query_id","title_id","prediction"]].to_csv('./submission/test2_lgb_baseline10.csv',index=None,header=0)

# 将ctr特征全部分出来，单独训练一个lgb模型

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import os
import gc
from sklearn.model_selection import train_test_split

In [ ]:
data_5 = pd.read_csv("/home/kesci/order_data/19.csv",names=["query_id","query","title_id","title","label"])

In [ ]:
# 只加载ctr相关的特征
temp_data=pd.read_csv("/home/kesci/work/features/train/ctr_5.csv")
data_5 = pd.concat([data_5,temp_data],axis=1)
temp_data=pd.read_csv("/home/kesci/train/query_title_word_ctr.csv")
data_5 = pd.concat([data_5,temp_data],axis=1)

In [ ]:
import pickle
all_title_C=pickle.load(open("/home/kesci/work/data/all_title_C.pkl","rb"))
def get_C(x):
    if x in all_title_C:
        return all_title_C[x]
    else:
        return 0
        
def get_I(x):
    if x in all_title_I:
        return all_title_I[x]
    else:
        return 0

# 得到每个title的展示次数
data_5["title_C"]=data_5["title"].apply(get_C)
del all_title_C
gc.collect()

all_title_I=pickle.load(open("/home/kesci/work/data/all_title_I.pkl","rb"))
# 得到每个title的点击次数
data_5["title_I"]=data_5["title"].apply(get_I)
del all_title_I
gc.collect()

In [ ]:
# 使用上面计算出来的平滑因子，得到title的ctr
data_5["title_ctr"]=(data_5["title_C"]+1.3912001443482402)/(data_5["title_I"]+1.3912001443482402+7.3260734359558795)

In [ ]:
train_colums=['5_ctr',
 'best_query_word_ctr',
 'best_title_word_ctr',
 'sum_query_word_ctr',
 'sum_title_word_ctr',
 'title_C',
 'title_I',
 'title_ctr']

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {"auc",'binary_logloss'},  #二进制对数损失
    'num_leaves': 80,
    'max_depth': 6,
     'subsample_freq':1,
     'feature_fraction':0.8,
    'learning_rate': 0.03,
     'reg_alpha':0.0,
     'reg_lambda':1,
}

X, val_X, y, val_y = train_test_split(
    train_x,
    train_y,
    test_size=0.001,
    random_state=1,
    stratify=train_y # 这里保证分割后y的比例分布与原数据一致
)
import gc
del train_x
del train_y
gc.collect()
X_train = X
y_train = y
X_test = val_X
y_test = val_y
lgb_train = lgb.Dataset(X_train,y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=5000,
                valid_sets=lgb_eval,
                early_stopping_rounds=100)
gbm.save_model('./model_file/ctr_lgb_mode2.txt')

# 得到最终测试集在ctr_lgb上的预测结果

In [ ]:
title_I_19=pickle.load(open("/home/kesci/title_I_19.pkl","rb"))
title_C_19=pickle.load(open("/home/kesci/title_C_19.pkl","rb"))

In [ ]:
def get_ctr_5(x):
    if x not in title_I_19:
        return 1.3912001443482402/(1.3912001443482402+7.3260734359558795)
    else:
        return (title_C_19[x]+1.3912001443482402)/(title_I_19[x]+1.3912001443482402+7.3260734359558795)

In [ ]:
data_test = pd.read_csv("/home/kesci/test2.csv",names=["query_id","query","title_id","title"])
data_test["5_ctr"]=data_test["title"].apply(get_ctr_5)
all_title_C=pickle.load(open("/home/kesci/work/data/all_title_C.pkl","rb"))

def get_C(x):
    if x in all_title_C:
        return all_title_C[x]
    else:
        return 0
        
def get_I(x):
    if x in all_title_I:
        return all_title_I[x]
    else:
        return 0

data_test["title_C"]=data_test["title"].apply(get_C)
del all_title_C
gc.collect()

all_title_I=pickle.load(open("/home/kesci/work/data/all_title_I.pkl","rb"))
data_test["title_I"]=data_test["title"].apply(get_I)
del all_title_I
gc.collect()

data_test["title_ctr"]=(data_test["title_C"]+1.3912001443482402)/(data_test["title_I"]+1.3912001443482402+7.3260734359558795)

In [ ]:
import pandas as pd
temp_data = pd.read_csv("/home/kesci/test2/query_title_word_ctr.csv")
data_test = pd.concat([data_test,temp_data],axis=1)

In [ ]:
test_x=data_test[train_colums].values

In [ ]:
gbm=lgb.Booster(model_file='./model_file/ctr_lgb_mode2.txt')
preds = gbm.predict(test_x, num_iteration=gbm.best_iteration)  


test_pred=pd.DataFrame(preds)
test_pred.columns=["prediction"]
test_pred["query_id"]=list(data_test["query_id"])
test_pred["title_id"]=list(data_test["title_id"])
test_pred[["query_id","title_id","prediction"]].to_csv('./submission/test2_ctr_lgb_pre2.csv',index=None,header=0)

# 神经网络模型ESIM，双端纯文本输入，未加特征

In [ ]:
# 加载并处理数据，对输入的文本进行编码处理，词转化成id，并对长度进行了对齐。
def load_esim_data_and_labels(data_path,char_to_id,q_max_len=22,t_max_len=22):
    f = open(data_path, 'r')
    y = []
    t_feat_index = []
    q_feat_index = []
    for line in f:
        line = line.strip().split(",")
        y.append([int(line[4])])
        query = []
        for i in line[1].split():
            if i in char_to_id:
                query.append(char_to_id[i])
            else:
                query.append(0)
        if len(query) < q_max_len:
            query = query + [0] * (q_max_len - len(query))
        else:
            query = query[:q_max_len]
        q_feat_index.append(query)
        title = []
        for i in line[3].split():
            if i in char_to_id:
                title.append(char_to_id[i])
            else:
                title.append(0)
        if len(title) < t_max_len:
            title = title + [0] * (t_max_len - len(title))
        else:
            title = title[:t_max_len]
        t_feat_index.append(title)
    f.close()
    return {"q_feat_index": q_feat_index,"t_feat_index": t_feat_index,"label": y}

# 一批一批加载并处理数据，对输入的文本进行编码处理，词转化成id，并对长度进行了对齐。
def yield_esim_data_and_labels(data_path,char_to_id,batch_size,q_max_len=22,t_max_len=22):
    f = open(data_path, 'r')
    y = []
    t_feat_index = []
    q_feat_index = []
    temp_b=0
    for line in f:
        line = line.strip().split(",")
        y.append([int(line[4])])
        query = []
        for i in line[1].split():
            if i in char_to_id:
                query.append(char_to_id[i])
            else:
                query.append(0)
        if len(query) < q_max_len:
            query = query + [0] * (q_max_len - len(query))
        else:
            query = query[:q_max_len]
        q_feat_index.append(query)
        title = []
        for i in line[3].split():
            if i in char_to_id:
                title.append(char_to_id[i])
            else:
                title.append(0)
        if len(title) < t_max_len:
            title = title + [0] * (t_max_len - len(title))
        else:
            title = title[:t_max_len]
        t_feat_index.append(title)
        temp_b += 1
        if temp_b == batch_size:
            yield {"q_feat_index": q_feat_index, "t_feat_index": t_feat_index, "label": y}
            y = []
            t_feat_index = []
            q_feat_index = []
            temp_b = 0
    if temp_b!=0:
        yield {"q_feat_index": q_feat_index,"t_feat_index": t_feat_index,"label": y}
    f.close()

In [ ]:
from torch import nn
import torch
import torch.nn.functional as F
import os
import numpy as np
from time import time
from torch.autograd import Variable
from gensim.models import Word2Vec
from sklearn.metrics import roc_auc_score
from gensim.models import FastText

word2id_file="./data/word2id.txt"
read_file = open(word2id_file, "r")
word2id={}
for i in read_file:
    i=i.strip().split()
    word2id[i[0]]=int(i[1])
read_file.close()

#参数配置
epcho=1
batch_size=256 
num_to_ev=400 # 训练多少批，在本地评测一次
vocab_size=len(word2id) # 词典大小
embedding_dim=256 # 词向量维度
t_max_len=22 #title的最大长度
q_max_len=11 #query的最大长度
lr=0.0001 #学习率



#加载验证集
val_data=load_esim_data_and_labels("/home/kesci/work/data/eval_data/19_eval.csv",word2id,q_max_len=q_max_len,t_max_len=t_max_len)


# 每个词拼接使用128维的w2v和128维的fast向量到256维的组合向量表示
ce = np.random.uniform(-1, 1, [vocab_size + 1,embedding_dim])
word2vec_model = Word2Vec.load("/home/kesci/word2vec.model")
fast_model = FastText.load("./data/fast_w2v.model")
ce[0] = np.zeros(embedding_dim)
for i in word2id:
    try:
        ce[word2id[i]] = np.concatenate((word2vec_model[i],fast_model[i]))
    except:
        print(i)

In [ ]:
# 注意力机制padding部分，赋予极小的数，这样算出来的padding部分注意力分数无限接近0，减少噪音。
mask_num=-2**32+1.0
# 模型结构，标准的esim结构
class ESIM(nn.Module):
    def __init__(self,hidden_size,embeds_dim,linear_size):
        super(ESIM, self).__init__()
        self.verbose=True
        self.dropout = 0.5
        self.n_epochs=epcho
        self.learning_rate=lr
        self.optimizer_type="adam"
        self.use_cuda=True
        self.batch_size=batch_size
        self.eval_metric=roc_auc_score
        self.hidden_size = hidden_size
        self.embeds_dim = embeds_dim
        num_word = vocab_size
        self.embeds = nn.Embedding(num_word+1, self.embeds_dim,padding_idx=0)
        self.embeds.weight.data.copy_(torch.from_numpy(ce)) # 使用ce矩阵，初始化embedding矩阵。

        self.bn_embeds = nn.BatchNorm1d(self.embeds_dim)
        self.lstm1 = nn.LSTM(self.embeds_dim, self.hidden_size, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(self.hidden_size*8, self.hidden_size, batch_first=True, bidirectional=True)

        self.fc = nn.Sequential(
            nn.BatchNorm1d(self.hidden_size * 8),
            nn.Linear(self.hidden_size * 8, linear_size),
            nn.ELU(inplace=True),
            nn.BatchNorm1d(linear_size),
            nn.Dropout(self.dropout),
            nn.Linear(linear_size, linear_size),
            nn.ELU(inplace=True),
            nn.BatchNorm1d(linear_size),
            nn.Dropout(self.dropout),
            nn.Linear(linear_size, 1)
        )
    
    def soft_attention_align(self, x1, x2, mask1, mask2):
        '''
        x1: batch_size * seq_len * dim
        x2: batch_size * seq_len * dim
        '''
        # attention: batch_size * seq_len * seq_len
        attention = torch.matmul(x1, x2.transpose(1, 2))
        mask1 = mask1.float().masked_fill_(mask1, mask_num)
        mask2 = mask2.float().masked_fill_(mask2, mask_num)

        # weight: batch_size * seq_len * seq_len
        weight1 = F.softmax(attention + mask2.unsqueeze(1), dim=-1)
        x1_align = torch.matmul(weight1, x2)
        weight2 = F.softmax(attention.transpose(1, 2) + mask1.unsqueeze(1), dim=-1)
        x2_align = torch.matmul(weight2, x1)
        # x_align: batch_size * seq_len * hidden_size

        return x1_align, x2_align

    def submul(self, x1, x2):
        mul = x1 * x2
        sub = x1 - x2
        return torch.cat([sub, mul], -1)

    def apply_multiple(self, x):
        # input: batch_size * seq_len * (2 * hidden_size)
        p1 = F.avg_pool1d(x.transpose(1, 2), x.size(1)).squeeze(-1)
        p2 = F.max_pool1d(x.transpose(1, 2), x.size(1)).squeeze(-1)
        # output: batch_size * (4 * hidden_size)
        return torch.cat([p1, p2], 1)

    def forward(self, *input):
        # batch_size * seq_len
        sent1, sent2 = input[0], input[1]
        mask1, mask2 = sent1.eq(0), sent2.eq(0)

        # embeds: batch_size * seq_len => batch_size * seq_len * dim
        x1 = self.bn_embeds(self.embeds(sent1).transpose(1, 2).contiguous()).transpose(1, 2)
        x2 = self.bn_embeds(self.embeds(sent2).transpose(1, 2).contiguous()).transpose(1, 2)

        # batch_size * seq_len * dim => batch_size * seq_len * hidden_size
        o1, _ = self.lstm1(x1)
        o2, _ = self.lstm1(x2)

        # Attention
        # batch_size * seq_len * hidden_size
        q1_align, q2_align = self.soft_attention_align(o1, o2, mask1, mask2)
        
        # Compose
        # batch_size * seq_len * (8 * hidden_size)
        q1_combined = torch.cat([o1, q1_align, self.submul(o1, q1_align)], -1)
        q2_combined = torch.cat([o2, q2_align, self.submul(o2, q2_align)], -1)

        # batch_size * seq_len * (2 * hidden_size)
        q1_compose, _ = self.lstm2(q1_combined)
        q2_compose, _ = self.lstm2(q2_combined)

        # Aggregate
        # input: batch_size * seq_len * (2 * hidden_size)
        # output: batch_size * (4 * hidden_size)
        q1_rep = self.apply_multiple(q1_compose)
        q2_rep = self.apply_multiple(q2_compose)

        # Classifier
        x = torch.cat([q1_rep, q2_rep], -1)
        similarity = self.fc(x)
        return similarity

    def fit(self,save_path = None):
        if save_path and not os.path.exists('/'.join(save_path.split('/')[0:-1])):
            print("Save path is not existed!")
            return

        if self.verbose:
            print("pre_process data ing...")
        if self.verbose:
            print("pre_process data finished")

        model = self.train()

        optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        if self.optimizer_type == 'adam':
            optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        elif self.optimizer_type == 'rmsp':
            optimizer = torch.optim.RMSprop(self.parameters(), lr=self.learning_rate)
        elif self.optimizer_type == 'adag':
            optimizer = torch.optim.Adagrad(self.parameters(), lr=self.learning_rate)

        criterion = F.binary_cross_entropy_with_logits
        max_eval=0.60
        valid_result = []
        for epoch in range(self.n_epochs):
            total_loss = 0.0
            for train_dict in yield_esim_data_and_labels("/home/kesci/shuffle_data.csv", word2id, num_to_ev*batch_size,
                                                        q_max_len=q_max_len,t_max_len=t_max_len):
                query=train_dict['q_feat_index']
                title=train_dict['t_feat_index']
                y_train=train_dict['label']
                query = np.array(query)
                y_train = np.array(y_train, dtype=np.float32)
                x_size = query.shape[0]
                batch_iter = x_size // self.batch_size
                epoch_begin_time = time()
                batch_begin_time = time()
                for i in range(batch_iter+1):
                    offset = i*self.batch_size
                    end = min(x_size, offset+self.batch_size)
                    if offset == end:
                        break
                    batch_query = Variable(torch.LongTensor(query[offset:end]))
                    batch_title = Variable(torch.LongTensor(title[offset:end]))

                    batch_y = Variable(torch.FloatTensor(y_train[offset:end]))
                    if self.use_cuda:
                        batch_query,batch_title,batch_y = batch_query.cuda(), batch_title.cuda(),batch_y.cuda()
                    optimizer.zero_grad()
                    #print(batch_query,batch_title)
                    outputs = model(batch_query,batch_title)
                    #print(outputs)
                    loss = criterion(outputs, batch_y)
                    loss.backward()
                    optimizer.step()
                    for p in model.parameters():
                        if p.grad is not None:
                            del p.grad  # free some memory
                    torch.cuda.empty_cache()
                    #model = self.train()

                    total_loss += loss.item()
                    if self.verbose:
                        if i % 100 == 99:
                            eval = self.evaluate(batch_query,batch_title,batch_y)
                            print('[%d, %5d] loss: %.6f metric: %.6f time: %.1f s' %
                                  (epoch + 1, i + 1, total_loss/100.0, eval, time()-batch_begin_time))
                            total_loss = 0.0
                            batch_begin_time = time()
                            for p in model.parameters():
                                if p.grad is not None:
                                    del p.grad  # free some memory
                            torch.cuda.empty_cache()
                            model = self.train()

                query_valid = val_data["q_feat_index"]
                title_valid = val_data["t_feat_index"]
                y_valid = val_data["label"]
                query_valid = np.array(query_valid)
                title_valid = np.array(title_valid)
                y_valid = np.array(y_valid,dtype=np.float32)
                x_valid_size = query_valid.shape[0]
                valid_loss, valid_eval = self.eval_by_batch(query_valid,title_valid,y_valid, x_valid_size)
                valid_result.append(valid_eval)
                print('*' * 50)
                print('[%d] loss: %.6f val_metric: %.6f time: %.1f s' %
                      (epoch + 1, valid_loss, valid_eval,time()-epoch_begin_time))
                print('*' * 50)
                if save_path and valid_eval>max_eval:
                    max_eval=valid_eval
                    torch.save(self, save_path)
                    print("保存模型成功")
                for p in model.parameters():
                    if p.grad is not None:
                        del p.grad  # free some memory
                torch.cuda.empty_cache()
                model = self.train()
                #torch.save(self.state_dict(),save_path)

    def eval_by_batch(self,query,title,y, x_size):
        total_loss = 0.0
        y_pred = []
        batch_size = self.batch_size
        batch_iter = x_size // batch_size
        criterion = F.binary_cross_entropy_with_logits
        model = self.eval()
        for i in range(batch_iter+1):
            offset = i * batch_size
            end = min(x_size, offset + batch_size)
            if offset == end:
                break
            batch_query = Variable(torch.LongTensor(query[offset:end]))
            batch_title = Variable(torch.LongTensor(title[offset:end]))
            batch_y = Variable(torch.FloatTensor(y[offset:end]))
            if self.use_cuda:
                batch_query,batch_title,batch_y = batch_query.cuda(),batch_title.cuda(),batch_y.cuda()
            
            with torch.no_grad():
                outputs = model(batch_query,batch_title)
                pred = torch.sigmoid(outputs).cpu()
                y_pred.extend(pred.data.numpy())
                loss = criterion(outputs, batch_y)
                total_loss += loss.item()*(end-offset)
        total_metric = self.eval_metric(y,y_pred)
        return total_loss/x_size, total_metric

    def predict_proba(self, batch_q,batch_t):
        batch_q = np.array(batch_q)
        batch_t = np.array(batch_t)
        batch_q = Variable(torch.LongTensor(batch_q))
        batch_t = Variable(torch.LongTensor(batch_t))
        if self.use_cuda and torch.cuda.is_available():
            batch_q = batch_q.cuda()
            batch_t = batch_t.cuda()

        model = self.eval()
        with torch.no_grad():
            pred = torch.sigmoid(model(batch_q,batch_t)).cpu()
        return pred.data.numpy()


    def inner_predict_proba(self,batch_q,batch_t):
        model = self.eval()
        with torch.no_grad():
            pred = torch.sigmoid(model(batch_q,batch_t)).cpu()
        return pred.data.numpy()


    def evaluate(self, batch_q,batch_t,y):
        y_pred = self.inner_predict_proba(batch_q,batch_t)
        return self.eval_metric(y.cpu().data.numpy(), y_pred)

In [ ]:
model = ESIM(hidden_size=256,embeds_dim=embedding_dim,linear_size=256).cuda()
model.fit(save_path="./model_file/esim_model.pkl")

# 使用ESIM模型生成最终测试集的结果

In [ ]:
#一批一批加载数据，到内存，对输入的文本进行编码处理，词转化成id，并对长度进行了对齐，并返回query_id和title_id
def yield_test_esim_data(data_path,char_to_id,batch_size,q_max_len=22,t_max_len=22):
    f = open(data_path, 'r')
    t_feat_index = []
    q_feat_index = []
    query_ids=[]
    title_ids=[]
    temp_b=0
    for line in f:
        line = line.strip().split(",")
        query = []
        for i in line[1].split():
            if i in char_to_id:
                query.append(char_to_id[i])
            else:
                query.append(0)
        if len(query) < q_max_len:
            query = query + [0] * (q_max_len - len(query))
        else:
            query = query[:q_max_len]
        q_feat_index.append(query)
        query_ids.append(line[0])
        title_ids.append(line[2])
        title = []
        for i in line[3].split():
            if i in char_to_id:
                title.append(char_to_id[i])
            else:
                title.append(0)
        if len(title) < t_max_len:
            title = title + [0] * (t_max_len - len(title))
        else:
            title = title[:t_max_len]
        t_feat_index.append(title)
        temp_b += 1
        if temp_b == batch_size:
            yield {"q_feat_index": q_feat_index, "t_feat_index": t_feat_index,"query_ids":query_ids,"title_ids":title_ids}
            t_feat_index = []
            q_feat_index = []
            query_ids=[]
            title_ids=[]
            temp_b = 0
    if temp_b!=0:
        yield {"q_feat_index": q_feat_index, "t_feat_index": t_feat_index,"query_ids":query_ids,"title_ids":title_ids}
    f.close()

In [ ]:
from torch import nn
import torch
import torch.nn.functional as F
import os
import numpy as np
from time import time
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score
import pandas as pd




word2id_file="./data/word2id.txt"
read_file = open(word2id_file, "r")
word2id={}
for i in read_file:
    i=i.strip().split()
    word2id[i[0]]=int(i[1])
read_file.close()

num_to_ev=100
batch_size=2048
vocab_size=len(word2id)
embedding_dim=256
t_max_len=22
q_max_len=11
lr=0.0003

In [ ]:
mask_num=-2**32+1.0
class ESIM(nn.Module):
    def __init__(self,hidden_size,embeds_dim,linear_size):
        super(ESIM, self).__init__()
        self.verbose=True
        self.dropout = 0.5
        self.n_epochs=epcho
        self.learning_rate=lr
        self.optimizer_type="adam"
        self.use_cuda=False
        self.batch_size=batch_size
        self.eval_metric=roc_auc_score
        self.hidden_size = hidden_size
        self.embeds_dim = embeds_dim
        num_word = vocab_size
        self.embeds = nn.Embedding(num_word+1, self.embeds_dim,padding_idx=0)

        self.bn_embeds = nn.BatchNorm1d(self.embeds_dim)
        self.lstm1 = nn.LSTM(self.embeds_dim, self.hidden_size, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(self.hidden_size*8, self.hidden_size, batch_first=True, bidirectional=True)

        self.fc = nn.Sequential(
            nn.BatchNorm1d(self.hidden_size * 8),
            nn.Linear(self.hidden_size * 8, linear_size),
            nn.ELU(inplace=True),
            nn.BatchNorm1d(linear_size),
            nn.Dropout(self.dropout),
            nn.Linear(linear_size, linear_size),
            nn.ELU(inplace=True),
            nn.BatchNorm1d(linear_size),
            nn.Dropout(self.dropout),
            nn.Linear(linear_size, 1)
        )
    
    def soft_attention_align(self, x1, x2, mask1, mask2):
        '''
        x1: batch_size * seq_len * dim
        x2: batch_size * seq_len * dim
        '''
        # attention: batch_size * seq_len * seq_len
        attention = torch.matmul(x1, x2.transpose(1, 2))
        mask1 = mask1.float().masked_fill_(mask1, mask_num)
        mask2 = mask2.float().masked_fill_(mask2, mask_num)

        # weight: batch_size * seq_len * seq_len
        weight1 = F.softmax(attention + mask2.unsqueeze(1), dim=-1)
        x1_align = torch.matmul(weight1, x2)
        weight2 = F.softmax(attention.transpose(1, 2) + mask1.unsqueeze(1), dim=-1)
        x2_align = torch.matmul(weight2, x1)
        # x_align: batch_size * seq_len * hidden_size

        return x1_align, x2_align

    def submul(self, x1, x2):
        mul = x1 * x2
        sub = x1 - x2
        return torch.cat([sub, mul], -1)

    def apply_multiple(self, x):
        # input: batch_size * seq_len * (2 * hidden_size)
        p1 = F.avg_pool1d(x.transpose(1, 2), x.size(1)).squeeze(-1)
        p2 = F.max_pool1d(x.transpose(1, 2), x.size(1)).squeeze(-1)
        # output: batch_size * (4 * hidden_size)
        return torch.cat([p1, p2], 1)

    def forward(self, *input):
        # batch_size * seq_len
        sent1, sent2 = input[0], input[1]
        mask1, mask2 = sent1.eq(0), sent2.eq(0)

        # embeds: batch_size * seq_len => batch_size * seq_len * dim
        x1 = self.bn_embeds(self.embeds(sent1).transpose(1, 2).contiguous()).transpose(1, 2)
        x2 = self.bn_embeds(self.embeds(sent2).transpose(1, 2).contiguous()).transpose(1, 2)

        # batch_size * seq_len * dim => batch_size * seq_len * hidden_size
        o1, _ = self.lstm1(x1)
        o2, _ = self.lstm1(x2)

        # Attention
        # batch_size * seq_len * hidden_size
        q1_align, q2_align = self.soft_attention_align(o1, o2, mask1, mask2)
        
        # Compose
        # batch_size * seq_len * (8 * hidden_size)
        q1_combined = torch.cat([o1, q1_align, self.submul(o1, q1_align)], -1)
        q2_combined = torch.cat([o2, q2_align, self.submul(o2, q2_align)], -1)

        # batch_size * seq_len * (2 * hidden_size)
        q1_compose, _ = self.lstm2(q1_combined)
        q2_compose, _ = self.lstm2(q2_combined)

        # Aggregate
        # input: batch_size * seq_len * (2 * hidden_size)
        # output: batch_size * (4 * hidden_size)
        q1_rep = self.apply_multiple(q1_compose)
        q2_rep = self.apply_multiple(q2_compose)

        # Classifier
        x = torch.cat([q1_rep, q2_rep], -1)
        similarity = self.fc(x)
        return similarity

    def eval_by_batch(self,query,title,y, x_size):
        total_loss = 0.0
        y_pred = []
        batch_size = self.batch_size
        batch_iter = x_size // batch_size
        criterion = F.binary_cross_entropy_with_logits
        model = self.eval()
        for i in range(batch_iter+1):
            offset = i * batch_size
            end = min(x_size, offset + batch_size)
            if offset == end:
                break
            batch_query = Variable(torch.LongTensor(query[offset:end]))
            batch_title = Variable(torch.LongTensor(title[offset:end]))
            batch_y = Variable(torch.FloatTensor(y[offset:end]))
            if self.use_cuda:
                batch_query,batch_title,batch_y = batch_query.cuda(),batch_title.cuda(),batch_y.cuda()
            
            with torch.no_grad():
                outputs = model(batch_query,batch_title)
                pred = torch.sigmoid(outputs).cpu()
                y_pred.extend(pred.data.numpy())
                loss = criterion(outputs, batch_y)
                total_loss += loss.item()*(end-offset)
        total_metric = self.eval_metric(y,y_pred)
        return total_loss/x_size, total_metric

    def predict_proba(self, batch_q,batch_t):
        batch_q = np.array(batch_q)
        batch_t = np.array(batch_t)
        batch_q = Variable(torch.LongTensor(batch_q))
        batch_t = Variable(torch.LongTensor(batch_t))
        if self.use_cuda and torch.cuda.is_available():
            batch_q = batch_q.cuda()
            batch_t = batch_t.cuda()

        model = self.eval()
        with torch.no_grad():
            pred = torch.sigmoid(model(batch_q,batch_t)).cpu()
        return pred.data.numpy()


    def inner_predict_proba(self,batch_q,batch_t):
        """
        :param Xi: tensor of feature index
        :param Xv: tensor of feature value
        :return: output, numpy
        """
        model = self.eval()
        with torch.no_grad():
            pred = torch.sigmoid(model(batch_q,batch_t)).cpu()
        return pred.data.numpy()


    def evaluate(self, batch_q,batch_t,y):
        """
        :param Xi: tensor of feature index
        :param Xv: tensor of feature value
        :param y: tensor of labels
        :return: metric of the evaluation
        """
        y_pred = self.inner_predict_proba(batch_q,batch_t)
        return self.eval_metric(y.cpu().data.numpy(), y_pred)

In [ ]:
esim_model = torch.load('/home/kesci/work/model_file/esim_model.pkl').cuda()

In [ ]:
outputfile=open("./submission/test2_esim_pre.csv","w")
all_ress=[]
all_len=0
for test_dict in yield_test_esim_data("/home/kesci/test2.csv", word2id, num_to_ev*batch_size,
                                                        q_max_len=q_max_len,t_max_len=t_max_len):
    ress=[]
    query = test_dict['q_feat_index']
    title = test_dict['t_feat_index']
    query = np.array(query)
    x_size = query.shape[0]
    batch_iter = x_size // batch_size

    for i in range(batch_iter + 1):
        offset = i * batch_size
        end = min(x_size, offset + batch_size)
        if offset == end:
            break
        batch_query = query[offset:end]
        batch_title = title[offset:end]
        res = esim_model.predict_proba(batch_query,batch_title)
        ress.extend(list([mm[0] for mm in res]))
        all_ress.extend(list([mm[0] for mm in res]))
    all_len+=x_size
    print(all_len)
    for dui in zip(test_dict["query_ids"],test_dict["title_ids"],ress):
        outputfile.write(str(dui[0])+","+str(dui[1])+","+str(dui[2])+"\n")
outputfile.close()

# 模型融合：简单将三个模型的结果文件，加和取平均即可。

In [ ]:
import pandas as pd
final_ctr_lgb_pre = pd.read_csv("/home/kesci/work/submission/test2_ctr_lgb_pre2.csv",
names=["q_id","t_id","pre"])
final_test2 = pd.read_csv('/home/kesci/work/submission/test2_esim_pre.csv',names=["q_id","t_id","pre"])
final_test3 = pd.read_csv('/home/kesci/work/submission/test2_lgb_baseline10.csv',names=["q_id","t_id","pre"])

In [ ]:
final_test3["final_pre"]=(final_ctr_lgb_pre["pre"]+final_test2["pre"]+final_test3["pre"])/3

In [ ]:
# test2_sub1.csv 即为最后的结果文件
final_test3[["q_id","t_id","final_pre"]].to_csv("/home/kesci/work/final_sub/test2_sub1.csv",
index=None,header=0)